<a href="https://colab.research.google.com/github/EoghanDoyle/Thesis/blob/main/Artefact_MovieLens_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619413 sha256=86cc2d4e5b637ce45ed02e36185a420b12e20e82bddb6e36424a7f54b0c9b626
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
#Imports
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection.split import train_test_split
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate
from surprise import NormalPredictor


In [ ]:
#-------------------------------------------------------Data Preprocessing -------------------------------------------------------

# Importing dataset and examining it
movies_dataset = pd.read_csv("/content/drive/My Drive/DBS/movies.csv")
# print(movies_dataset.head())
# print(movies_dataset.shape)
# print(movies_dataset.info())
# print(movies_dataset.describe())

ratings_dataset = pd.read_csv("/content/drive/My Drive/DBS/ratings.csv")
# print(ratings_dataset.head())
# print(ratings_dataset.shape)
# print(ratings_dataset.info())
# print(ratings_dataset.describe())

#merge 2 tables 
dataset = ratings_dataset.merge(movies_dataset,on='movieId', how='left')
print(dataset)

#drop unnecessary columns
refined_dataset = dataset.drop(['timestamp','genres','title'],axis=1)
print(refined_dataset.head())

#gets total numbers of ratings for each movie
total_ratings = pd.DataFrame(refined_dataset.groupby('movieId')['rating'].count())
print(total_ratings.head(10))

total_ratings.to_csv('/content/drive/My Drive/DBS/MovieLens_total_ratings.csv')

#merge ratings df and original df
combined_dataset = refined_dataset.merge(total_ratings,on='movieId', how='left')
print(combined_dataset.head())

min_movie_ratings = 2 #a movie has was rated at least 
min_user_ratings =  5 #a user rated movies at least

                                                                                                                                                                                                                                    
movies_filtered = combined_dataset.groupby("movieId").filter(lambda x: x['movieId'].count() >= min_movie_ratings)
movies_filtered = movies_filtered.groupby("userId").filter(lambda x: x['userId'].count() >= min_user_ratings)

final_movies_dataset = movies_filtered.drop(['rating_y'],axis=1)

reader = Reader()
#dataset creation
data = Dataset.load_from_df(final_movies_dataset, reader)


        userId  ...                                       genres
0            1  ...  Adventure|Animation|Children|Comedy|Fantasy
1            1  ...                               Comedy|Romance
2            1  ...                        Action|Crime|Thriller
3            1  ...                             Mystery|Thriller
4            1  ...                       Crime|Mystery|Thriller
...        ...  ...                                          ...
100831     610  ...                        Drama|Horror|Thriller
100832     610  ...                        Action|Crime|Thriller
100833     610  ...                                       Horror
100834     610  ...                                Action|Sci-Fi
100835     610  ...                  Action|Crime|Drama|Thriller

[100836 rows x 6 columns]
   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0
         rating
movieId        

In [ ]:
#------------------------------------------------------------------ KNN --------------------------------------------------------
param_grid = {'k':[10,20,30,40,50,60] , 
              'sim_options': {'name': ['cosine', 'pearson'] }
              }

gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])
print("Best MAE score attained: ", gs.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
print(gs.best_params['mae'])

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computi

In [ ]:
#------------------------------------------------------------------ SVD ---------------------------------------------------------
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])
print("Best MAE score attained: ", gs.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
print(gs.best_params['mae'])


Best RMSE score attained:  0.8661363966658149
Best MAE score attained:  0.6630009129744017
{'n_epochs': 30, 'lr_all': 0.005, 'n_factors': 50}
{'n_epochs': 30, 'lr_all': 0.005, 'n_factors': 50}


In [ ]:
#------------------------------------------------------------------ SVD++ ---------------------------------------------------------
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])
print("Best MAE score attained: ", gs.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
print(gs.best_params['mae'])


Best RMSE score attained:  0.8564539756033683
Best MAE score attained:  0.6571536659099013
{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}
{'n_epochs': 30, 'lr_all': 0.005, 'n_factors': 50}


In [ ]:
#------------------------------------------------------------------ Random ---------------------------------------------------------
Random = NormalPredictor()
results = cross_validate(Random, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
print(results)


{'test_rmse': array([1.41362941, 1.42174574, 1.40787943, 1.41527353, 1.41053732]), 'test_mae': array([1.13144761, 1.1345179 , 1.12605281, 1.12915295, 1.12816007]), 'fit_time': (0.13488507270812988, 0.16836142539978027, 0.16152667999267578, 0.16358613967895508, 0.16169428825378418), 'test_time': (0.16257357597351074, 0.16051769256591797, 0.14049768447875977, 0.2979738712310791, 0.1419391632080078)}
